In [1]:
!pip install -U sentence-transformers
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from ast import literal_eval
import chromadb
from tqdm.auto import tqdm

# Load dataset

In [3]:
df = pd.read_csv("/content/25k-imdb-movie-dataset.csv")

In [4]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


# Data processing

In [5]:
def concatenate_list(my_list):
    my_list = literal_eval(my_list)
    return " ".join(my_list)

In [6]:
df = df.fillna(" ")
df["Keywords"] = df["Plot Kyeword"].apply(concatenate_list)
df["Stars"] = df["Top 5 Casts"].apply(concatenate_list)
df["Generes"] = df["Generes"].apply(concatenate_list)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").fillna(0).astype("float")

In [7]:
df.drop(["Plot Kyeword", "Top 5 Casts"], axis=1, inplace=True)

In [8]:
df["text"] = df.apply(lambda x : str(x["Overview"])+" "+x["Keywords"]+" "+x["Stars"], axis=1)

In [9]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,Action Drama,After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,Action Adventure Sci-Fi,Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,Action Drama,As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...


# Load model

In [10]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Generate vector embeddings

In [11]:
embeddings = model.encode(df["text"], batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/382 [00:00<?, ?it/s]

In [12]:
df["embeddings"] = embeddings.tolist()
df["ids"] = df.index
df["ids"] = df["ids"].astype("str")

# Connect to ChromaDB

In [13]:
chroma_client = chromadb.PersistentClient(path="/content/data_embeddings")

In [14]:
db = chroma_client.create_collection(name="movies_db")

# Populate ChromaDB database

In [15]:
batch_size = 500

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i + batch_size, len(df))
    batch = df.iloc[i:i_end]

    ids = batch["ids"].tolist()
    emb = batch["embeddings"].tolist()
    metadata = batch.drop(["ids", "embeddings", "text", "path"], axis=1).to_dict("records")

    db.add(
        ids=ids,
        embeddings=emb,
        metadatas=metadata
    )

  0%|          | 0/49 [00:00<?, ?it/s]

# Semantic search

In [16]:
def search(query, genre, rating, max_results):
    # Generate embedding from query
    query_vector = model.encode(query).tolist()

    filter_rating = rating

    if genre:
        if isinstance(genre, str):
            genre = [genre]
        genre = [g.lower() for g in genre]
    else:
        genre = []

    # Query
    results = db.query(
        query_embeddings=[query_vector],
        n_results=max_results * 2,
        include=["documents", "metadatas", "distances"]
    )

    # Apply filters
    response_data = []
    for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        # Filter by genre
        if genre and not any(g in meta.get("Generes", "").lower() for g in genre):
            continue
        # Filter by rating
        if float(meta.get("Rating", 0)) < filter_rating:
            continue

        # Append results into list
        response_data.append({
            "Title": meta.get("movie title"),
            "Overview": meta.get("Overview"),
            "Director": meta.get("Director"),
            "Genre": meta.get("Generes"),
            "Year": meta.get("year"),
            "Rating": meta.get("Rating"),
            "Score": dist
        })

    df = pd.DataFrame(response_data[:max_results])

    return df

In [21]:
responses = search(
    query="a story of time travel",
    genre=["Action", "Sci-Fi", "comedy"],
    rating=6.0,
    max_results=5
)

responses

,Title,Overview,Director,Genre,Year,Rating,Score
0,24,"A scientist invents a time machine, which lead...",Vikram K. Kumar,Action Comedy Drama,(I) (2016),7.9,0.745679
1,Dikkiloona,A man travels back in time to change his unple...,Karthik Yogi,Comedy Romance Sci-Fi,-2021,6.3,0.803258
2,Aditya 369,A group of children use a time machine to inve...,Singeetam Srinivasa Rao,Adventure Comedy Sci-Fi,-1991,8.5,0.818203
3,Back to the Future,"Marty McFly, a 17-year-old high school student...",Robert Zemeckis,Adventure Comedy Sci-Fi,-1985,8.5,0.827078
